# Plotly-dash
[참고] https://dash.plot.ly/  <br>
[참고2] https://dash.plot.ly/dash-core-components <br>
[참고3] https://dash.plot.ly/interactive-graphing

## Part2. Dash Layout
* plotly dash 실행 서버
    * http://127.0.0.1:8050/

> Dash구조
* layout은 html.Div 와 dcc.Graph로 구성되어 있음.
* dash_html_components 라이브러리는 모든 HTML태그의 컴포넌트가 들어있다. 
* dash_core_components 는 인터렉티브한 하이레벨의 컴포넌트를 제공. React.js 라이브러리기반
* children 프로퍼티는 특별하다. 
    * html.H1(children='Hello Dash') 와 html.H1('Hello Dash') 은 같다.

> 구성요소
* app.run_server(debug=True, use_reloader=False) 은 사용자가 코드를 변경했을 때 자동으로 Dash를 refresh 해주는 기능

> Visualization에 대한
* dash_core_components라이브러리는 Graph라 불리는 컴포넌트를 갖고 있다.
    * Graphh 는 plotly.js 오픈소스를 사용해서 인터렉티브한 데이터 시각화를 렌더해준다 .
* figure 는 plotly.py에서 사용되는 figure와 같은 기능. 

> Markdown
* dash_core_components 라이브러리의 Markdown 컴포넌트 사용. <br>
* 예시 <br>
markdown_text = ''' <br>
마크다운 내용 입력 <br>
''' <br>
app.layout = html.Div([<br>
    dcc.Markdown(children=markdown_text)<br>
])<br>

## Part3. Basic Dash Callbacks

> Dash Layout
* app.callback 을 통해서 input과 output 가 표현된다.
* Dash에서 input과 output은 특정 컴포넌트의 속성이다. 
* conponent_id 와 component_property 키워드는 옵션이다.

    => 새로운 텍스트를 띄우기 위해서는 children을 업데이트하고, 새로운 데이터를 띄우기 위해서는 dcc.Graph 의 figure를 업데이트한다.


### [ error사항 해결방법 ]
* 아래처럼 use_reloader=False 값 추가해야됨.

if __name__ == '__main__':  <br>
    app.run_server(debug=True, use_reloader=False)<br>
    
    
    

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
plt.style.use('ggplot')
#plt.style.use('fivethirtyeight')
import scipy as sp
import re

In [3]:
# warning 무시
import warnings
warnings.filterwarnings(action='ignore')

### 데이터 불러오기

In [18]:
# 날짜 데이터 불러올 때 
df=pd.read_excel('./data/raw_data_세부.xlsx',encoding='euc-kr',parse_dates={'날짜':["년","월"]})
df=df.rename(columns={'날짜':'date','분류1':'device','분류2':'shopping_type','서비스분류1':'service','종류1':'type1','종류2':'type2','값':'value'})

# 종합_쇼핑기여 제외 (종합 만 )
df=df[df['shopping_type']=='종합']
#df.head(10)

# 광고 타입 (직접광고, 네트워크 ...) 별 매출
## 서비스 전체매출
data_adtype_sales=df.query('type2!=["PV","UV","일평균방문횟수"] and device=="PC" and type1 == "매출 구분"')
data_adtype_sales=data_adtype_sales.dropna(subset=['value'])        # 'value'컬럼에 NA있을때 제거

# service컬럼에 '뉴스' 또는 '기사박스' 일경우 '뉴스'로 통일. 
## TV,스푼피드,여행,이글루스,자동차,증권정보,코인,허브,이슈트렌드는 모두 '서브'로 통일
data_adtype_sales['service_group']=np.where(data_adtype_sales['service'].isin(['기사박스']),'뉴스',data_adtype_sales['service'] )
data_adtype_sales['service_group']=np.where(data_adtype_sales['service'].isin(['TV','스푼피드','여행','이글루스','자동차','증권정보','코인','허브','이슈트렌드']),'서브',data_adtype_sales['service_group'])
#print(data_adtype_sales['service_group'].unique())

# 광고구분없이 서비스별(프런트, 검색, 뉴스, 서브) 매출 (groupby sum)
data_adtype_sales_groupsum=pd.DataFrame(data_adtype_sales.groupby(['date','service_group'], as_index=False)['value'].sum()) 

# PC, 서비스별 트래픽(PV,UV) 데이터
data_traffic=df.query('device=="PC" and type1=="트래픽" and (type2=="UV" or type2=="PV") ')[['date', 'device','service','type1','type2','value']]
data_traffic['service_group']=np.where(data_traffic['service'].isin(['기사박스']),'뉴스',data_traffic['service'])
data_traffic['service_group']=np.where(data_traffic['service'].isin(['TV','스푼피드','여행','이글루스','자동차','증권정보','코인','허브','이슈트렌드']),'서브',data_traffic['service_group'])

data_traffic_groupsum=pd.DataFrame(data_traffic.groupby(['date','service_group','type2'],as_index=False)['value'].sum())

sales =data_adtype_sales.groupby(['date','service_group']).sum()
sales_pcts=pd.DataFrame(round((sales/sales.groupby(level=0).sum())*100,2).reset_index()['value'])
data_adtype_sales_groupsum=pd.concat([data_adtype_sales_groupsum, sales_pcts],axis=1)
data_adtype_sales_groupsum.columns=['date','service_group','value','pcts']

data_adtype_sales_groupsum['inc_pcts']=round(((sales.groupby(level=1).pct_change())*100).reset_index()['value'],2)

# 서비스별, 광고종류별 매출
### bar-plot으로 총매출 그리고, line-plot으로 직접광고, 쇼핑, 네트워크, 제휴 각각 그리기
### 프런트, 검색, 뉴스, 서브 각각 다른 subplot 4개로
data_adtype_sales_groupsum2=pd.DataFrame(data_adtype_sales.groupby(['date','service_group','type2'],as_index=False)['value'].sum())

# 프론트 트래픽
data_traffic_front=df.query('device=="PC" and service=="프런트" and type1=="트래픽" and (type2=="UV" or type2=="PV") ')[['date', 'device','service','type1','type2','value']]

#  PC, 트래픽 평균단가
data_traffic_price=df.query('device=="PC" and type1=="트래픽 평균단가"')
data_traffic_price= data_traffic_price.dropna(subset=['value'])

# 서비스 카테고리 4개로 축소 : 프런트, 검색, 뉴스, 서브
data_traffic_price['service_group']=np.where(data_traffic_price['service'].isin(['기사박스']),'뉴스', data_traffic_price['service'])
data_traffic_price['service_group']=np.where(data_traffic_price['service'].isin(['TV','스푼피드','여행','이글루스','자동차','증권정보','코인','허브','이슈트렌드']),'서브',data_traffic_price['service_group'])

# 소수점 
data_traffic_price['value']=round(data_traffic_price['value'],2)

data_traffic_price_groupsum=pd.DataFrame(data_traffic_price.groupby(['date','service_group','type2'])['value'].sum().reset_index())

from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

In [4]:
# 날짜 데이터 불러올 때 
df=pd.read_excel('./data/raw_data_세부.xlsx',encoding='euc-kr',parse_dates={'날짜':["년","월"]})
df=df.rename(columns={'날짜':'date','분류1':'device','분류2':'shopping_type','서비스분류1':'service','종류1':'type1','종류2':'type2','값':'value'})

# 종합_쇼핑기여 제외 (종합 만 )
df=df[df['shopping_type']=='종합']
#df.head(10)

----------------------------------
## Dash-plot
#### PC-서비스별-트래픽-매출

In [15]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='KPI-Dash'),

    html.Div(children='''
        Dash: zum traffic and sales visualization
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': list(data_traffic_groupsum.query('service_group=="프런트"and type2=="PV"')['date']),
                 'y':list(data_traffic_groupsum.query('service_group=="프런트"and type2=="PV"')['value']), 
                 'type': 'line', 
                 'name': '프런트_PV'},
                
                  {'x': list(data_traffic_groupsum.query('service_group=="검색"and type2=="PV"')['date']),
                 'y':list(data_traffic_groupsum.query('service_group=="검색"and type2=="PV"')['value']), 
                 'type': 'line', 
                 'name': '검색_PV'},
                
                {'x': list(data_traffic_groupsum.query('service_group=="뉴스"and type2=="PV"')['date']),
                 'y':list(data_traffic_groupsum.query('service_group=="뉴스"and type2=="PV"')['value']), 
                 'type': 'line', 
                 'name': '뉴스_PV'},
                
                {'x': list(data_traffic_groupsum.query('service_group=="서브"and type2=="PV"')['date']),
                 'y':list(data_traffic_groupsum.query('service_group=="서브"and type2=="PV"')['value']), 
                 'type': 'line', 
                 'name': '서브_PV'},
                
                ### 매출
                {'x': list(data_adtype_sales_groupsum.query('service_group=="프런트"')['date']), 
                 'y': list(data_adtype_sales_groupsum.query('service_group=="프런트"')['value']), 
                 'type': 'bar', 
                 'name': '프런트_매출'},
                
                {'x': list(data_adtype_sales_groupsum.query('service_group=="검색"')['date']), 
                 'y': list(data_adtype_sales_groupsum.query('service_group=="검색"')['value']), 
                 'type': 'bar', 
                 'name': '검색_매출'},
                
                {'x': list(data_adtype_sales_groupsum.query('service_group=="뉴스"')['date']), 
                 'y': list(data_adtype_sales_groupsum.query('service_group=="뉴스"')['value']), 
                 'type': 'bar', 
                 'name': '뉴스_매출'},
                                
                {'x': list(data_adtype_sales_groupsum.query('service_group=="서브"')['date']), 
                 'y': list(data_adtype_sales_groupsum.query('service_group=="서브"')['value']), 
                 'type': 'bar', 
                 'name': '서브_매출'},
                
            ],
            'layout': {
                'title': 'PC-서비스별-트래픽-매출'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 100-907-916
Debugger PIN: 100-907-916
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


----------------------------
### Tab 기능 구현
[참고] https://dash.plot.ly/dash-core-components   <br>
[참고2] https://dash.plot.ly/dash-core-components/tabs

[ Tab기능 구현 방법 2가지 ]
1. Content as Callback
    * Tabs의 value 속성에 callback을 붙이고 컨테이너의 children 속성을 업데이트한다.
2. Content as Tab Children
    * callback을 통해서 content를 보여주는거대신, Tab 컴포넌트의 children속성으로 임베딩할수있다.

In [84]:
import dash
import dash_html_components as html
import dash_core_components as dcc

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
markdown_text = '''
### KPI-Dash board
PC와 Mobile로 구분되어 있음.

<탭별 설명>
* PC_전체
* PC_매출구성
* PC_트래픽
'''
markdown_tab1='''

'''


app.layout = html.Div([
    dcc.Markdown(children=markdown_text),
    
    dcc.Tabs([
########################################################################################## Tab1        
        dcc.Tab(label='PC_전체', children=[
            dcc.Markdown(children=markdown_text)
            dcc.Graph(
                figure={
                    'data': [
                        #### PV
                        {'x': list(data_traffic_groupsum.query('service_group=="프런트"and type2=="PV"')['date']), 
                         'y': list(data_traffic_groupsum.query('service_group=="프런트"and type2=="PV"')['value']),
                            'type': 'line', 
                             'marker':{'color':'#B0122C'},
                             'name': '프런트_PV'
                        },
                        {'x': list(data_traffic_groupsum.query('service_group=="검색"and type2=="PV"')['date']), 
                         'y': list(data_traffic_groupsum.query('service_group=="검색"and type2=="PV"')['value']),
                            'type': 'line', 
                             'marker':{'color':'royalblue'},
                             'name': '검색_PV'
                        },
                        {'x': list(data_traffic_groupsum.query('service_group=="뉴스"and type2=="PV"')['date']), 
                         'y': list(data_traffic_groupsum.query('service_group=="뉴스"and type2=="PV"')['value']),
                            'type': 'line', 
                             'marker':{'color':'orangered'},
                             'name': '뉴스_PV'
                        },
                        {'x': list(data_traffic_groupsum.query('service_group=="서브"and type2=="PV"')['date']), 
                         'y': list(data_traffic_groupsum.query('service_group=="서브"and type2=="PV"')['value']),
                            'type': 'line', 
                             'marker':{'color':'green'},
                             'name': '서브_PV'
                        },
                        
                        #### 매출
                        {'x': list(data_adtype_sales_groupsum.query('service_group=="프런트"')['date']),
                         'y': list(data_adtype_sales_groupsum.query('service_group=="프런트"')['value']),
                         'type': 'bar', 
                         'marker':{'color':'rgba(220,49,72,0.8)'},
                         'name': '프런트_매출'
                        },
                        {'x': list(data_adtype_sales_groupsum.query('service_group=="검색"')['date']),
                         'y': list(data_adtype_sales_groupsum.query('service_group=="검색"')['value']),
                         'type': 'bar', 
                         'marker':{'color':'LightSkyBlue'},
                         'name': '검색_매출'
                        },
                        {'x': list(data_adtype_sales_groupsum.query('service_group=="뉴스"')['date']),
                         'y': list(data_adtype_sales_groupsum.query('service_group=="뉴스"')['value']),
                         'type': 'bar', 
                         'marker':{'color':'orange'},
                         'name': '뉴스_매출'
                        },
                        {'x': list(data_adtype_sales_groupsum.query('service_group=="서브"')['date']),
                         'y': list(data_adtype_sales_groupsum.query('service_group=="서브"')['value']),
                         'type': 'bar', 
                         'marker':{'color':'limegreen'},
                         'name': '서브_매출'
                        },
                    ],
                     'layout': {
                         'title': '매출 기본',
                         'height':500,
                         'width':1000
                       #  'margin': {'l': 20, 'b': 20, 'r': 20, 't': 20}
                     }
                }
            )
        ]),
        
########################################################################################## Tab2
        dcc.Tab(label='PC_매출구성', children=[
            dcc.Graph(
                figure={
                    'data': [
                        {'x': list(data_adtype_sales_groupsum2.query('service_group=="프런트"and type2=="총매출"')['date']), 
                         'y': list(data_adtype_sales_groupsum2.query('service_group=="프런트"and type2=="총매출"')['value']),
                         'type': 'bar', 
                         'marker':{'color':'lightskyblue'},
                         'name': '프런트_총매출'
                        },
                        #### 광고 종류별 매출
                        {'x': list(data_adtype_sales_groupsum2.query('service_group=="프런트"and type2=="네트워크"')['date']),
                         'y': list(data_adtype_sales_groupsum2.query('service_group=="프런트"and type2=="네트워크"')['value']),
                         'type': 'line', 
                         'marker':{'color':'lightseagreen'},
                         'name': '프런트_네트워크'
                        },
                        {'x': list(data_adtype_sales_groupsum2.query('service_group=="프런트"and type2=="직접광고"')['date']),
                         'y': list(data_adtype_sales_groupsum2.query('service_group=="프런트"and type2=="직접광고"')['value']),
                         'type': 'line', 
                         'marker':{'color':'olive'},
                         'name': '프런트_직접광고'
                        },
                        {'x': list(data_adtype_sales_groupsum2.query('service_group=="프런트"and type2=="쇼핑"')['date']),
                         'y': list(data_adtype_sales_groupsum2.query('service_group=="프런트"and type2=="쇼핑"')['value']),
                         'type': 'line', 
                         'marker':{'color':'mediumblue'},
                         'name': '프런트_쇼핑'
                        },
                        {'x': list(data_adtype_sales_groupsum2.query('service_group=="프런트"and type2=="제휴"')['date']),
                         'y': list(data_adtype_sales_groupsum2.query('service_group=="프런트"and type2=="제휴"')['value']),
                         'type': 'line', 
                         'marker':{'color':'mediumpurple'},
                         'name': '프런트_제휴'
                        },
                    ],
                    'layout': {
                         'title': '프런트-영역별 종합 매출 및 매출 구성',
                         'height':500,
                         'width':1000
                     }
                }
            ),
            dcc.Graph(
                figure={
                    'data': [
                        {'x': list(data_adtype_sales_groupsum2.query('service_group=="검색"and type2=="총매출"')['date']), 
                         'y': list(data_adtype_sales_groupsum2.query('service_group=="검색"and type2=="총매출"')['value']),
                         'type': 'bar', 
                         'marker':{'color':'lightskyblue'},
                         'name': '검색_총매출'
                        },
                        #### 광고 종류별 매출
                        {'x': list(data_adtype_sales_groupsum2.query('service_group=="검색"and type2=="제휴"')['date']),
                         'y': list(data_adtype_sales_groupsum2.query('service_group=="검색"and type2=="제휴"')['value']),
                         'type': 'line', 
                         'marker':{'color':'mediumpurple'},
                         'name': '검색_제휴'
                        },
                    ],
                    'layout': {
                         'title': '검색-영역별 종합 매출 및 매출 구성',
                         'height':500,
                         'width':1000
                     }
                }
            )
        ]),
        
########################################################################################## Tab3
        dcc.Tab(label='PC_트래픽', children=[
            dcc.Graph(
                figure={
                    'data': [
                        {'x': list(data_traffic_groupsum.query('service_group=="프런트"and type2=="PV"')['date']), 
                         'y': list(data_traffic_groupsum.query('service_group=="프런트"and type2=="PV"')['value']),
                            'type': 'line', 'name': '프런트_PV'
                        },
                        {'x': list(data_traffic_groupsum.query('service_group=="검색"and type2=="PV"')['date']), 
                         'y': list(data_traffic_groupsum.query('service_group=="검색"and type2=="PV"')['value']),
                            'type': 'line', 'name': '검색_PV'
                        },
                    ]
                }
            )
        ]),
    ])
])


if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on htt

#### dropdown선택 값에 따라 아래 dropdown값 달라지는 기능
https://dash.plot.ly/persistence